In [6]:
## VISSIM Modules
import win32com.client as com
import os
## FA Modules
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("ERROR: GPU DEVICE NOT FOUND.")

# Import Keras is TF 1.    
#from keras.models import load_model

import sys
sys.path.append('../')

## Data Management Modules
import pickle
from collections import defaultdict
## Math Modules
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
import math

Default GPU Device: /device:GPU:0


# Define a Signal Control Unit

Internals can change depending on setup but I would like there to be methods:

update() 

action_update()

sar() 


In [7]:
# SCU


'''
Signal_Control_Unit :
    
    interfaces between a signal controller (at a junction) and actions (provided by an agent)
    
    inputs:
    -- Vissim
    -- Signal_Controller - a Vissim, signal controller
    -- compatible_actions - a dictionary taking IDs to vectors non-conflicting signal groups
    -- green_time = 10   (times are in seconds but converted to simulation steps)
    -- redamber_time = 1
    -- amber_time = 3
    -- red_time = 1
     
    methods :
    -- action_update():
            initiates the change to a new action
            
            inputs:
            -- action_key 
            -- green_time 
    -- update():
            ensures each signal is correct at each simulation step
'''
class Signal_Control_Unit:
    
    def __init__(self,\
                 Vissim,\
                 Signal_Controller,\
                 compatible_actions,\
                 Signal_Groups = None,\
                 green_time = 40,\
                 redamber_time = 1,\
                 amber_time = 3, \
                 red_time = 1\
                ):
        
        # get Vissim, signal controller and its signal groups
        self.Vissim = Vissim
        self.signal_controller = Signal_Controller
        
        if Signal_Groups is None :
            self.signal_groups = self.signal_controller.SGs
        else :
            self.signal_groups = Signal_Groups
            
        # get stae and reward parameters
        self.state = self.calculate_state()
        self.reward = self.calculate_reward()  
       
        self.compatible_actions = compatible_actions
          
        self.time_steps_per_second = self.Vissim.Simulation.AttValue('SimRes')
        
        self.green_time = green_time * self.time_steps_per_second
        self.redamber_time = redamber_time * self.time_steps_per_second
        self.amber_time = amber_time * self.time_steps_per_second
        self.red_time = red_time * self.time_steps_per_second
    
        # implement 1st action to start
        self.action_key = 0   # dict key of current action (we start with 0) 
        self.action_required = False # used to requests an action from agent
        self.update_counter = 0
        self.intermediate_phase = True # tracks when initiating a new action
        self.action_update(self.action_key)    
        

        self.stage = "Green" # tracks the stage particularly when in intermediate phase.
                             # Stages appear in order: "Amber" -> "Red" -> "RedAmber" -> "Green"

            
    '''
    sars :
    returns state, id of action, reward
    '''     
    def sar(self):
        self.state = self.calculate_state()
        self.reward = self.calculate_reward()
        
        return self.state, self.action_key, self.reward

    
    '''
    calculate_state:
    Alvaro's reward function needs to be more general
    '''
    def calculate_state(self,length=None,verbose = False):
        
        Queues = []
        Lanes = []
        for sg in self.signal_groups :
            q = 0 
            for sh in sg.SigHeads:
                if (sh.Lane.AttValue('Link'),sh.Lane.AttValue('Index')) not in Lanes :
                    Lanes.append((sh.Lane.AttValue('Link'),sh.Lane.AttValue('Index')))
                    for veh in sh.Lane.Vehs:
                        q += veh.AttValue('InQueue')
            Queues.append(q)
            # Summarize queue size in each lane
            if verbose :
                print(self.signal_controller.AttValue('No'),sg.AttValue('No'),q)
            
        # now reshape
        if length is not None :
            state = np.reshape(Queues,[1,length])
        else :
            state = np.reshape(Queues,[1,len(Queues)])
        
        return (state)

    
    '''
    calculate_reward:
    Alvaro's reward function needs to be more general
    '''
    def calculate_reward(self):
        state = self.calculate_state()
        reward = -np.sum(state)
        
        return reward
 

    '''
    action_update :
    initiates a new action
        inputs:
        -- id of action
        -- green_time, if specified by agent (in seconds)
    '''    
    def action_update(self,action_key,green_time=None):
        self.intermediate_phase = True # initate intermediate_phase
        self.update_counter = 0 # set update counter zero (will get reset at self.update() )
        self.action_key = action_key
        self.current_action = self.compatible_actions[action_key] 
        self.new_colors = [ 2*val for val in self.current_action] # converts action to 0,1,2 range
        
        if green_time is not None:
            self.green_time = green_time * self.time_steps_per_second
        

    # internal helper function
    # red = 0, amber/redamber = 1 and green = 2
    def _color_convert(self,color):
        if color == "RED" :
            return 0
        elif color == "GREEN" :
            return 2
        else :
            return 1

        
    '''
    _color_changer :
    Internal function
    Changes color of a signal group
        inputs:
        -- signal group
        -- new_color : 2 = green / 0 = red
        -- stage : what stage all lights in the controller are.
    '''          
    def _color_changer(self,signal_group,new_color,stage):
        #Get the current color
        current_color = self._color_convert(signal_group.AttValue("SigState"))
        change = new_color-current_color
        
        # want green but currently red
        if change == -2 and stage == "Green" :
            signal_group.SetAttValue("SigState", "AMBER")
        
        # want red but currently amber
        # if just gone red need on second before green change
        elif change == -1 and stage == "Amber" :
            signal_group.SetAttValue("SigState", "RED")
        
        # want green but currently red 
        elif change == 2 and stage == "Red" :
            signal_group.SetAttValue("SigState", "REDAMBER")
                
        # want green but currently redamber
        elif change == 1 and stage == "RedAmber":
            signal_group.SetAttValue("SigState", "GREEN")
        
        # if both red or green pass (i.e. no change keep green)
        elif change == 0 :
            pass
    

    '''
    _stage_changer :
    Internal function
    
    Track controllers stage (in the stages of Amber->Red->RedAmber-Green) 
    and time for each transtion
    
        inputs:
        -- stage
        
    Nb. stage is a controller method while color is a sg property
    '''
    def _stage_changer(self,stage):
        
        if stage == "Green" :
            time = self.amber_time
            self.stage = "Amber" 
    
        elif stage == "Amber" :
            time = self.red_time
            self.stage = "Red"
        
        elif stage == "Red" :
            time = self.redamber_time
            self.stage = "RedAmber"
                
        # want green but currently redamber
        elif stage == "RedAmber" :
            time = self.green_time
            self.stage = "Green"
        
        return time
                
        
    '''
    update :
    
    returns True if action required (otherwise is None)
    
    implements cycle at each signal group
    and updates the stage of the controllers.
    
    (writen so multiple controllers can be updated in parallel)
    (Computational Overhead should be lower than before)
    
    '''   
    def update(self):
    
        self.update_counter -= 1
        
        # These 'if' clauses mean update computation only happens if needed
        if self.update_counter <= 0 :
            # if update counter just went zero 
            # then ask for an action 
            if self.intermediate_phase is False :
                self.action_required = True 
                return self.action_required
                    
            # if during a change
            # then make the change
            if self.intermediate_phase is True : 
                self.action_reqired = False
                
                # Get light color right for each signal group
                for sg in self.signal_groups :
                    ID = sg.AttValue('No')-1
                    self._color_changer(sg,self.new_colors[ID],self.stage)
                        
                # change the current stage and get time the stage last for
                time = self._stage_changer(self.stage)
                self.update_counter = time
                    
                # if full transition (Amber->Red->RedAmber-Green) to green done  
                if self.stage == "Green" :
                    self.intermediate_phase = False # record current action is implemented
                 
    

### Define a simple agent

In [8]:
class Cyclic_Control():
    def __init__(self,size):
        self.action = 0
        self.size = size
        
    def choose_action(self,state=None):
        self.action = (self.action + 1) % self.size
        return self.action

## Load Vissim and collect together network parameters 

This collection of parameters could be done with a 

network parser 

and

com_dispatch 

functions

In [9]:
# Load Vissim
for _ in range(5):
    try:
        Vissim = com.Dispatch("Vissim.Vissim")
        print('success!')
        break
    except:
        print('fail')

fail
fail
success!


In [10]:
# Load Network
#input_file = 'C:\\Users\\nwalton\\OneDrive - The Alan Turing Institute\\Documents\\MLforFlowOptimisation\\Vissim\\Balance\\Balance.inpx'
#input_file = 'C:\\Users\\Public\\Documents\\PTV Vision\\PTV Vissim 11\\Examples Training\\Signal Control\\UTC - Workflow PTV Balance PTV Epics\\03 AFTER In PTV Vissim\\PTV Balance PTV Epics Vision Suite Workflow.inpx'
input_file = 'C:\\Users\\nwalton\\OneDrive - The Alan Turing Institute\\Documents\\MLforFlowOptimisation\\Vissim\\NSW\\Balance_NSW\\Balance.inpx'
Vissim.LoadNet(input_file)
#Env = SE.Q_Sim_Env(Vissim,model_name,vissim_working_directory)

In [11]:
# all controller actions
Controllers_Actions =\
{\
    # Controller Number 2 
    0 : {   0 : [ 1, 0, 0, 1, 0, 0,],
            1 : [ 0, 1, 0, 0, 1, 0,],
            2 : [ 0, 0, 1, 0, 0, 1,]
        },
    # Controller Number 3
    1 : {   0 : [ 1, 0, 0, 1, 0, 0,],
            1 : [ 0, 1, 0, 0, 1, 0,],
            2 : [ 0, 0, 1, 0, 0, 1,]
        },
    # Controller Number 4
    2 : {
            0 : [1,0,0,0,1,0,0,0],
            1 : [0,1,0,0,0,1,0,0],
            2 : [0,0,1,0,0,0,1,0],
            3 : [0,0,0,1,0,0,0,1]
        },
    # Controller Number 5
    3 : {
            0 : [1,0,0,1,0,0],
            1 : [0,1,0,0,1,0],
            2 : [0,0,1,0,0,1],
        },
    # Controller Number 6 
    4 : {
            0 : [1,1,0,0],
            1 : [0,1,1,0],
            2 : [1,0,0,1]
        },
    # Controller Number 8
    5 : {
            0 : [1,0,1],
            1 : [0,1,0]
        },
    # Controller Number 9
    6 : {
            0 : [1,0,1,0],
            1 : [0,1,0,1]
        },
    # Contoller Number 10
    7 : {
            0 : [1,0,0,0,0],
            1 : [0,1,0,1,0],
            2 : [0,0,1,0,1]
        },
    # Controller Number 12
    8 : {
            0 : [1,0,1],
            1 : [0,1,0],
        },
    # Controller Number 13
    9 : {
            0 : [1,0,0,0],
            1 : [0,1,0,1],
            1 : [0,0,1,0],
        },
    # Controller 15
     10 : {   
            0 : [1,0,1,0],\
            1 : [0,1,0,1]\
        },
    # Controller 16
    11 : {   
            0 : [1,0,1,0],\
            1 : [0,1,0,1]\
        },
    # Controller 17
    12 : {
            0 : [1,0,0,0,0],
            1 : [0,1,0,0,1],
            2 : [0,0,1,0,0],
            3 : [0,0,0,1,1],
            4 : [0,1,0,0,1]
        },
    # Controller 33 
    13 : {
            0 : [1,0,0,1],
            1 : [1,0,1,0],
            2 : [0,1,0,0]
        }
}

In [12]:
number_of_scs = len(Controllers_Actions)

Signal_Controller = []
for idx in range(number_of_scs):
    Signal_Controller.append(Vissim.Net.SignalControllers[idx])

Signal_Groups = [ [] for _ in range(number_of_scs)]

for i, SGs in enumerate(Signal_Groups) :  
    for j in range(len(Controllers_Actions[i][0])):
        SGs.append(Signal_Controller[i].SGs[j])
        #print(Signal_Controller[i].SGs[j].AttValue('No'))

SCU =  []

for i in range(number_of_scs):
    SCU.append(\
              Signal_Control_Unit(
                 Vissim,\
                 Signal_Controller[i],\
                 Controllers_Actions[i],\
                 Signal_Groups = Signal_Groups[i],\
                 green_time = 5,\
                 redamber_time = 1,\
                 amber_time = 3, \
                 red_time = 1\
                )\
              )

CC =[]
for i, acts in enumerate(Controllers_Actions):
    cycle_size = len(Controllers_Actions[i])
    CC.append(Cyclic_Control(cycle_size))

# This is the main control look

In [13]:
# Main loop
for _ in range(1000):  
    Vissim.Simulation.RunSingleStep()
   # time.sleep(0.3)
    for i, scu in enumerate(SCU) :
        action_required = scu.update()
        if action_required :
            '''
            Comments would be added for to an agent that actually 'learns'
            '''
            # state = scu.calculate_state()
            # sars = scu.sars
            # Agent[i].Learn(sars)
            # new_action = Agent[i].choose_action(state)
            new_action = CC[i].choose_action()
            scu.action_update(new_action)

## Other Code

Here is some code for an 

agent class 
simulation environment class (single junction)
simulation enviroment for multiple agents


In [ ]:
##################################
## THIS IS A GENERIC SIMULATION ##
##################################

class Simulation_Environment():
    def __init__(self,\
                 Loaded_Vissim,\
                 sim_length=3601,\
                 timesteps_per_second=12):
        
        self.Vissim = Loaded_Vissim
        self.Quick_Mode(Quick_Mode)
            
        
    def Step(self,action,sim_steps=10):
        
        state = _calculate_state(self)
        reward = _calculate_reward(self)
    
        return  state, reward, done
        
    def _calculate_state(self):
        pass
    
    def _calculate_reward(self):
        pass
    
#    def Quick_Mode(self,Quick_Mode):
#        self.Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode",Quick_Mode)
        
# Queue Length Based Simulator
class Q_Sim_Env(Simulation_Environment):
    def __init__(self,\
                 Loaded_Vissim,\
                 model_name,\
                 vissim_working_directory,\
                 Lane_List = ['3-1','1-1','7-1','5-1'],\
                 sim_length=3601,\
                 timesteps_per_second=1,\
                 delete_results=True,\
                 verbose=False):
    
        # Load init from Parent Class
        super(Q_Sim_Env,self).__init__(
                 Loaded_Vissim,\
                 model_name,\
                 vissim_working_directory,\
                 sim_length,\
                 timesteps_per_second,\
                 delete_results,\
                 verbose)

    
        self.Signal_Controller = self.Vissim.Net.SignalControllers.GetAll()[0]
        self.Signal_Groups = self.Signal_Controller.SGs.GetAll()
        self.Lane_List = Lane_List

    def Step(self,action=None,Signal_Groups=None):
        if Signal_Groups is None:
            Signal_Groups = self.Signal_Groups
    # Consist of 4 steps: 
    # Greens go Amber
    # Ambers go Red
    # Reds go RedAmber
    # RedAmbers go Green

        # Initial Parameters
        Sim_Period = self.Vissim.Simulation.AttValue('SimPeriod') #End of Simulation
        Amber_Time = 4. #One second of Amber
        Red_Time = 1.
        RedAmber_Time = 1.

        # If current_state = 'GREEN' and next_state = 'RED'
        # Then go AMBER        
        for i, sg in enumerate(Signal_Groups):
            current_state = sg.AttValue("SigState")
            if current_state == "GREEN" and action[i] == 0 :
                sg.SetAttValue("SigState", "AMBER")

        # Simulate 4 seconds for Amber
        Sim_Time = self.Vissim.Simulation.AttValue('SimSec')
        Amber_Break = min(Sim_Time+Amber_Time,Sim_Period)
        self.Vissim.Simulation.SetAttValue('SimBreakAt', Amber_Break)
        self.Vissim.Simulation.RunContinuous()

        # Set the AMBER lights red
        for i, sg in enumerate(Signal_Groups):
            current_state = sg.AttValue("SigState")
            if current_state == "AMBER":
                sg.SetAttValue("SigState", "RED")

        # Simulate 1 second for Red
        Sim_Time = self.Vissim.Simulation.AttValue('SimSec')
        Red_Break = min(Sim_Time+Red_Time,Sim_Period)
        self.Vissim.Simulation.SetAttValue('SimBreakAt', Red_Break)
        self.Vissim.Simulation.RunContinuous()

        # If current state "RED" and next_state = "GREEN"
        # Then go RedAmber
        for i, sg in enumerate(Signal_Groups):
            current_state = sg.AttValue("SigState")
            if current_state == "RED" and action[i] == 1 :
                sg.SetAttValue("SigState", "REDAMBER")

        # Simulate 1 second for RedAmber
        Sim_Time = self.Vissim.Simulation.AttValue('SimSec')
        RedAmber_Break = min(Sim_Time+RedAmber_Time,Sim_Period)
        self.Vissim.Simulation.SetAttValue('SimBreakAt', RedAmber_Break)
        self.Vissim.Simulation.RunContinuous()

        # Finally set all RedAmbers to Green
        for i, sg in enumerate(Signal_Groups):
            current_state = sg.AttValue("SigState")
            if current_state == "REDAMBER":
                sg.SetAttValue("SigState", "GREEN")
                
        state = self._calculate_state()
        reward = self._calculate_reward()
        not_finished = self.Vissim.Simulation.AttValue('IsRunning')
        done = False if not_finished == 1 else True
        
        return  state, reward, done
                
    def _calculate_state(self,Lane_List=None, rounding=1.):
        # Loads globals if variables not specfied
        if Lane_List is None :
            Lane_List = self.Lane_List

        # initialize with zero queues
        Qsum = 0
        Q_sizes = dict.fromkeys(Lane_List)
        for key in Q_sizes.keys():
            Q_sizes[key]=0

        # initialize with zero numbers of non-waiting cars
        nonQsum = 0
        nonQ_sizes = dict.fromkeys(Lane_List)
        for key in nonQ_sizes.keys():
            nonQ_sizes[key]=0

        # get all Q lengths    
        All_Vehicles = self.Vissim.Net.Vehicles.GetAll() 
        for Veh in All_Vehicles:
            lane = Veh.AttValue('Lane')
            if lane in Lane_List : 
                if Veh.AttValue('InQueue') == 1 :
                    Q_sizes[lane] += 1
                else : 
                    nonQ_sizes[lane] += 1

        state = []

        for lane in Lane_List :
            state.append(math.ceil(Q_sizes[lane] / rounding))

        return np.reshape(state, [1,len(state)])
    
    def _calculate_reward(self,Q_Size=None):
        # Use global as default
        if Q_Size is None:
            Q_Size = self._calculate_state()

        return -np.sum(Q_Size)

In [14]:
from collections import defaultdict
import numpy as np
#############################
## THIS IS A GENERIC AGENT ##
#############################
class Agent():     
    # Initialize agent with dimension of state and action space
    def __init__(self,state_size, action_size,actions):
        # Number of states, action space and memory
        self.state_size = state_size
        self.action_size = action_size
        self.actions = actions

    # Choose and action
    def Action(self, state, actions=None):
        if actions is None:
            actions = self.actions
        pass
    
    # Learning routine
    def Learn(self,sarsa):
        pass
    
    # learn from a batch
    def Learn_Batch(self,Sarsa, batch_size=32):
        pass
    
'''
MaxWeight Agent
'''
# MaxWeight Agent 
class MaxWeight(Agent):
    def Action(self,state,actions=None):
        if actions is None:
            actions=self.actions
            
        opt_val = 0
        for act in actions : 
            val = np.dot(act,state)
            if val >= opt_val :
                opt_val = val
                opt_act = act
        return opt_act
    
    
'''
Easy Q_learner
'''
class Q_function(Agent):
    def __init__(self, actions = None):
        # Q function
        self.Q = defaultdict(lambda: defaultdict(float))
        # number of visits
        self.N = defaultdict(lambda: defaultdict(float))
        self.actions = actions

    def Check(self,state,actions=None):
        if actions is None :
            actions = self.actions
        
        if state not in self.Q.keys():
            for action in actions:
                self.Q[state][action] = 0

    def Max(self,state):
        Q_maximum = np.max(list(self.Q[state].values()))
        return Q_maximum

    def Action(self,state,epsilon=0):
        if np.random.rand() < epsilon :
            idx = np.random.randint(len(actions))
            action = actions[idx]
        else :
            self.Check(state,actions)
            action = max(self.Q[state], key=self.Q[state].get)
        return action

    def Learn(self,sars,learning_rate=0.1,discount_factor=0.5):
        state, action, reward, next_state = sars
        # Check if state,action and next_state are in Q
        self.Check(state)
        self.Check(next_state)
        self.N_update(state,action)

        dQ = reward \
            + discount_factor * self.Max(next_state) \
            - self.Q[state][action]
        self.Q[state][action] = self.Q[state][action] + learning_rate * dQ 
        
        return self.Q

    def N_update(self,state,action,actions=None):
        if actions is None :
            actions = self.actions
        
        if state not in self.N.keys():
            for action in actions:
                self.N[state][action] = 0 
        self.N[state][action] = self.N[state][action] + 1
        return self.N[state][action]

    def Print(self):
        for state in Q_fn.Q.keys():
            for action in Q_fn.Q[state].keys():
                print(state,action,Q_fn.N[state][action],Q_fn.Q[state][action])

In [15]:
# Environment Class
'''
THIS IS A WORK IN PROGRESS MULTIAGENT SIMULATION ENVIRONMENT
'''

class Env_MultiAgent(Simulation_Environment):
    def __init__(self,\
                 Loaded_Vissim,\
                 Controllers_Actions,\
                 Signal_Controllers=None,\
                 sim_length=3601,\
                 timesteps_per_second=1,\
                ):
    
        # Load init from Parent Class
        super(Env_MultiAgent,self).__init__(
                 Loaded_Vissim,\
                 sim_length,\
                 timesteps_per_second)
        

        self.Vissim = Loaded_Vissim


        #self.Quick_Mode(Quick_Mode)
        
        # Signal_Controllers is all controllers or a specified list    
        if Signal_Controllers is not None :
            self.Signal_Controllers = Signal_Controllers
        else : 
            self.Signal_Controllers = Vissim.Net.SignalControllers
        
        self.Controllers_Actions = Controllers_Actions 
        self.SCUs = self._Load_SCUs()
        
    '''
    _Load_SCUs :
        provides a dictionary with at the SCUs
    '''
    def _Load_SCUs(self):
        
        SCUs = dict()
        
        for idx, sc in enumerate(self.Signal_Controllers):
            SCUs[idx] = Signal_Control_Unit(Vissim, sc, self.Controllers_Actions[idx])
        
        return SCUs
    
    '''
    Make a Step
    '''   
    def Step(self,action):
        self.Vissim.Simulation.RunSingleStep()
        
        Sars = dict()
        
        for idx, scu in self.SCUs:
            scu.action_required = SCU.update()
            if action_required :
                Sars[idx] = scu.sars()
        
        if len(actions_required) > 0 :
            return True, Sars
        else:
            return False, None
    
    def sar(self):
        sar_all = dict()
        
        for idx, scu in self.SCUs:
            sar_all[idx] = (scu._calculate_state, scu.action_key, scu._calculate_reward) 
        
        return sar_all
        
    def _calculate_state(self):
        '''
        THIS SHOULD GET QUEUE LENGTH FOR EACH Signal Control Group
        Debug because of multiple signal heads
        '''

        pass
    
#     def _calculate_reward(self):
#         pass
    
#     def Quick_Mode(self,Quick_Mode):
#         self.Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode",Quick_Mode)

NameError: name 'Simulation_Environment' is not defined